In [7]:
from openai import OpenAI
import os
from groq import Groq
import copy
import configs
import pickle
import re

os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

In [4]:
response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다."},
            {"role": "user", "content":
                f"""
                당신의 현재 좌표는 (131,134) 이며 당신의 목표 지점은 (23,200) 입니다.
                서쪽은 -x축에 대응합니다
                동쪽은 +x축에 대응합니다
                남쪽은 -y축에 대응합니다
                북쪽은 +y축에 대응합니다

                1. 당신의 목표 지점은 당신으로부터 어느 방향에 있는지 한 단어로 말하시오.
                
                Example: {{south}}, {{southwest}}, {{west}}, {{northwest}}, {{north}}, {{northeast}}, {{east}}, {{southeast}}
                
                당신은 가로로 긴 창고 환경에 있기 때문에, y축으로 이동이 가능할 때는 y축을 먼저 이용하는 것이 강력하게 추천됩니다. (남쪽과 북쪽)
                당신의 동쪽은 벽으로 막혀 있습니다.
                당신의 서쪽은 열려 있습니다.
                당신의 북쪽은 열려 있습니다.
                당신의 남쪽은 열려 있습니다.

                당신은 한 번에 한 번만 움직일 수 있습니다.
                당신은 동쪽, 서쪽, 남쪽, 북쪽, 그리고 가만히 있기 5가지의 선택지가 있습니다.
                따라서, 당신의 답은 east, west, south, north, stay 중 하나가 됩니다.

                2. 당신은 다음 어느 방향으로 움직일지 한 단어로 말하시오.

                Example: {{east}}, {{west}}, {{south}}, {{north}}, {{stay}}

                아래는 바람직한 답변의 예시입니다. 토큰을 많이 쓰지 마세요.
                1. {{southeast}}
                2. {{south}}
                """
            }],
        )

result = response.choices[0].message.content

In [5]:
lines = result.split('\n')

for line in lines:
    match = re.search(r'\d+\.\s*(\w+)', line)
    if match:
        direction = match.group(1).lower()

direction

'north'

In [11]:
response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다."},
            {"role": "user", "content":
                f"""
                당신의 지난 방문한 좌표들은 다음과 같습니다.
                오른쪽으로 갈수록 당신이 최근에 방문한 좌표입니다.
                [(0,1), (0, 2), (0, 3), (0, 4), (0, 5)]

                1. 여기서 만약 계속 반복되는 좌표가 있다면, 그 좌표를 지정하고 dead end라는 답을 내세요. 아니라면, no라는 답을 내세요.

                당신의 목표 지점은 당신의 북서쪽 방향에 있습니다.
                
                당신은 가로로 긴 창고 환경에 있기 때문에, y축으로 이동이 가능할 때는 y축을 먼저 이용하는 것이 강력하게 추천됩니다. (남쪽과 북쪽)
                당신의 동쪽은 벽으로 막혀 있습니다.
                당신의 서쪽은 열려 있습니다.
                당신의 북쪽은 열려 있습니다.
                당신의 남쪽은 열려 있습니다.

                당신은 한 번에 한 번만 움직일 수 있습니다.
                당신은 동쪽, 서쪽, 남쪽, 북쪽, 그리고 가만히 있기 5가지의 선택지가 있습니다.

                당신은 다음 어느 방향으로 움직일지 결정하세요.

                아래는 답변의 예시입니다. 토큰을 많이 사용하지 마세요.
                1. no
                2. north

                1. dead end at (2, 3)
                2. south
                """
            }],
        )

result = response.choices[0].message.content

lines = result.split('\n')

# 첫 번째 줄과 두 번째 줄을 변수에 할당
result1 = lines[0]
result2 = lines[1]

if "dead end" in result1:
    print(result1.split(" at ")[1])
else:
    print(result2.split(". ")[1])

north


In [13]:
(2, 3)[1]

3

In [83]:
response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다."},
            {"role": "user", "content":
                f"""
                당신은 dead end에서 똑같은 위치를 반복하고 있는 상태입니다.

                반복되는 좌표의 위치는 당신 기준 northwest입니다.

                당신은 dead end로부터 최대한 멀리 떨어져야 합니다.

                당신의 행동에는 east, west, south, north의 선택지가 있습니다.

                1. 당신은 다음 어느 방향으로 움직일지 결정하세요.

                아래는 답변의 예시입니다. Answer with only one word.
                south
                """
            }],
        )

result = response.choices[0].message.content

result

'east'

In [26]:
client = Groq(api_key="")

In [1]:
response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다."},
            {"role": "user", "content":
                f"""
                당신의 현재 좌표는 (131,134) 이며 당신의 목표 지점은 (23,200) 입니다.
                서쪽은 -x축에 대응합니다
                동쪽은 +x축에 대응합니다
                남쪽은 -y축에 대응합니다
                북쪽은 +y축에 대응합니다

                1. 당신의 목표 지점은 당신으로부터 어느 방향에 있는지 한 단어로 말하시오.
                
                Example: south, southwest, west, northwest, north, northeast, east, southeast
                
                당신은 가로로 긴 창고 환경에 있기 때문에, y축으로 이동이 가능할 때는 y축을 먼저 이용하는 것이 강력하게 추천됩니다. (남쪽과 북쪽)
                당신의 동쪽은 벽으로 막혀 있습니다.
                당신의 서쪽은 열려 있습니다.
                당신의 북쪽은 열려 있습니다.
                당신의 남쪽은 열려 있습니다.

                당신은 한 번에 한 번만 움직일 수 있습니다.
                당신은 동쪽, 서쪽, 남쪽, 북쪽, 그리고 가만히 있기 5가지의 선택지가 있습니다.
                따라서, 당신의 답은 east, west, south, north, stay 중 하나가 됩니다.

                2. 당신은 다음 어느 방향으로 움직일지 한 단어로 말하시오.

                Example: east, west, south, north, stay
                """
            }],
        )

response.choices[0].message.content

NameError: name 'client' is not defined

In [12]:
import openai
import concurrent.futures

# OpenAI API 키 설정

# Chat Completion을 수행할 함수 정의
def get_chat_completion(a):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다."},
            {"role": "user", "content":
                f"""
                당신의 현재 좌표는 (131,134) 이며 당신의 목표 지점은 (23,200) 입니다.
                서쪽은 -x축에 대응합니다
                동쪽은 +x축에 대응합니다
                남쪽은 -y축에 대응합니다
                북쪽은 +y축에 대응합니다

                1. 당신의 목표 지점은 당신으로부터 어느 방향에 있는지 한 단어로 말하시오.
                
                Example: {{south}}, {{southwest}}, {{west}}, {{northwest}}, {{north}}, {{northeast}}, {{east}}, {{southeast}}
                
                당신은 가로로 긴 창고 환경에 있기 때문에, y축으로 이동이 가능할 때는 y축을 먼저 이용하는 것이 강력하게 추천됩니다. (남쪽과 북쪽)
                당신의 동쪽은 벽으로 막혀 있습니다.
                당신의 서쪽은 열려 있습니다.
                당신의 북쪽은 열려 있습니다.
                당신의 남쪽은 열려 있습니다.

                당신은 한 번에 한 번만 움직일 수 있습니다.
                당신은 동쪽, 서쪽, 남쪽, 북쪽, 그리고 가만히 있기 5가지의 선택지가 있습니다.
                따라서, 당신의 답은 east, west, south, north, stay 중 하나가 됩니다.

                2. 당신은 다음 어느 방향으로 움직일지 한 단어로 말하시오.

                Example: {{east}}, {{west}}, {{south}}, {{north}}, {{stay}}

                아래는 바람직한 답변의 예시입니다. 토큰을 많이 쓰지 마세요.
                1. {{southeast}}
                2. {{south}}
                """
            }],
        )
    
    return a, response.choices[0].message.content

# 동시에 수행할 프롬프트 목록
prompts = [
    "What is the capital of France?",
    "Explain the theory of relativity.",
    "How does a blockchain work?",
    "What are the benefits of meditation?",
    "Give me a recipe for a chocolate cake."
]

# concurrent.futures를 이용해 동시에 Chat Completion 수행
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # 각 프롬프트에 대해 비동기 작업 제출
    futures = [executor.submit(get_chat_completion, prompt) for prompt in prompts]
    
    results = []

    # 각 작업의 결과 출력
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        results.append(result)

In [13]:
results

[('What is the capital of France?', '1. northwest\n2. north'),
 ('Give me a recipe for a chocolate cake.', '1. northwest\n2. north'),
 ('How does a blockchain work?', '1. {northwest}\n2. {north}'),
 ('What are the benefits of meditation?', '1. northwest\n2. north'),
 ('Explain the theory of relativity.', '1. northwest\n2. north')]

In [14]:
asdf = [(3, 4), (5, 6), (7, 8)]
for (a, b) in asdf:
    print(a)

3
5
7
